In [12]:
import boto3
from botocore.exceptions import BotoCoreError, ClientError
from contextlib import closing
import os
import sys
import subprocess
from tempfile import gettempdir

## AWS credentials and service selection

In [4]:
## Selecting profile credentials
session = boto3.session.Session(profile_name='aws_academy')

## Selecting AWS Service
polly = session.client(service_name='polly')

## Languages Supported by Amazon Polly [docs](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/polly.html#Polly.Client.describe_voices)

In [27]:
response = polly.describe_voices(LanguageCode='en-US')

In [28]:
response

{'ResponseMetadata': {'RequestId': '0f6fd411-7616-4ff8-95a8-5b9241b69758',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '0f6fd411-7616-4ff8-95a8-5b9241b69758',
   'content-type': 'application/json',
   'content-length': '1559',
   'date': 'Thu, 07 Apr 2022 15:46:09 GMT'},
  'RetryAttempts': 0},
 'Voices': [{'Gender': 'Female',
   'Id': 'Salli',
   'LanguageCode': 'en-US',
   'LanguageName': 'US English',
   'Name': 'Salli',
   'SupportedEngines': ['neural', 'standard']},
  {'Gender': 'Female',
   'Id': 'Joanna',
   'LanguageCode': 'en-US',
   'LanguageName': 'US English',
   'Name': 'Joanna',
   'SupportedEngines': ['neural', 'standard']},
  {'Gender': 'Male',
   'Id': 'Kevin',
   'LanguageCode': 'en-US',
   'LanguageName': 'US English',
   'Name': 'Kevin',
   'SupportedEngines': ['neural']},
  {'Gender': 'Male',
   'Id': 'Matthew',
   'LanguageCode': 'en-US',
   'LanguageName': 'US English',
   'Name': 'Matthew',
   'SupportedEngines': ['neural', 'standard']},
  {'Ge

## SynthesizeSpeech [docs](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/polly.html#Polly.Client)

In [22]:
text_1 = "El viejo y el mar es una novela escrita por Ernest Hemingway en 1951 en Cuba y publicada en 1952"

text_2 = "Legendary leadership and elite performance expert Robin Sharma introduced The 5am Club concept over twenty years ago"

In [30]:
try:
    # Request speech synthesis
    response = polly.synthesize_speech(Text=text_2, OutputFormat="mp3", VoiceId="Matthew")
except (BotoCoreError, ClientError) as error:
    # The service returned an error, exit gracefully
    print(error)
    sys.exit(-1)

In [31]:
response

{'ResponseMetadata': {'RequestId': '79be8a45-1625-4b1b-81d7-e27c2fbac451',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '79be8a45-1625-4b1b-81d7-e27c2fbac451',
   'x-amzn-requestcharacters': '116',
   'content-type': 'audio/mpeg',
   'transfer-encoding': 'chunked',
   'date': 'Thu, 07 Apr 2022 15:46:43 GMT'},
  'RetryAttempts': 0},
 'ContentType': 'audio/mpeg',
 'RequestCharacters': 116,
 'AudioStream': <botocore.response.StreamingBody at 0x7fa623237d00>}

In [32]:
# Access the audio stream from the response
if "AudioStream" in response:
    # Note: Closing the stream is important because the service throttles on the
    # number of parallel connections. Here we are using contextlib.closing to
    # ensure the close method of the stream object will be called automatically
    # at the end of the with statement's scope.
        with closing(response["AudioStream"]) as stream:
           output = os.path.join(gettempdir(), "speech.mp3")

           try:
            # Open a file for writing the output as a binary stream
                with open(output, "wb") as file:
                   file.write(stream.read())
           except IOError as error:
              # Could not write to file, exit gracefully
              print(error)
              sys.exit(-1)

else:
    # The response didn't contain audio data, exit gracefully
    print("Could not stream audio")
    sys.exit(-1)

# Play the audio using the platform's default player
if sys.platform == "win32":
    os.startfile(output)
else:
    # The following works on macOS and Linux. (Darwin = mac, xdg-open = linux).
    opener = "google-chrome" # "open" if sys.platform == "darwin" else "xdg-open"
    #print(opener)
    #print(output)
    subprocess.call([opener, output])